<a href="https://colab.research.google.com/github/Sushmithaa-Pandian/NLP/blob/main/SpamorHam_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [19]:
!pip install tensorflow_text

In [20]:
import os
import tensorflow as tf
import tensorflow_hub as hub
!pip install transformers
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
import tensorflow as tf
from tensorflow import keras
from keras import layers
import pandas as pd

In [21]:
import tensorflow_text

In [24]:
df=pd.read_csv("/content/gdrive/MyDrive/spam_or_not_spam.csv")
df.head(5)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['email'],df['label'], stratify=df['label'])

bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")


In [25]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding([
    "we lied down around am rose",
    "i hate it i want my dj"]
)

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.8089319 , -0.2035183 ,  0.43780452, ...,  0.33894002,
        -0.5936639 ,  0.88505703],
       [-0.8107354 , -0.22443093,  0.26202512, ...,  0.36520654,
        -0.5653348 ,  0.8948848 ]], dtype=float32)>

In [26]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [27]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 keras_layer_6 (KerasLayer)  {'input_type_ids': (None,    0         ['text[0][0]']                
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_word_ids': (None,                                           
                              128)}                                                               
                                                                                            

In [32]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [38]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
71/71 [==============================] - 1384s 19s/step - loss: 0.4488 - accuracy: 0.8338
Epoch 2/5
71/71 [==============================] - 1348s 19s/step - loss: 0.4144 - accuracy: 0.8351
Epoch 3/5
71/71 [==============================] - 1323s 19s/step - loss: 0.3887 - accuracy: 0.8373
Epoch 4/5
 4/71 [>.............................] - ETA: 20:34 - loss: 0.4233 - accuracy: 0.8125

KeyboardInterrupt: ignored

In [34]:
print(len(X_train))

2250


In [37]:
import numpy as np
X_train = np.array([np.array(val) for val in X_train])
y_train = np.array([np.array(val) for val in y_train])


In [39]:
model.evaluate(X_test, y_test)

24/24 [==============================] - 445s 18s/step - loss: 0.3637 - accuracy: 0.8400


[0.36372220516204834, 0.8399999737739563]

In [40]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()
print(y_predicted)

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

24/24 [==============================] - 448s 19s/step
[0.13330491 0.11298487 0.08613058 0.15419696 0.21823312 0.45975932
 0.10599633 0.1654583  0.15954249 0.17613037 0.3752574  0.16371448
 0.11607898 0.1406386  0.10783073 0.1032217  0.11134025 0.2648652
 0.18025151 0.16327502 0.10182435 0.4311775  0.14822766 0.2479255
 0.11640458 0.19888982 0.20844357 0.10267411 0.24764006 0.17157197
 0.13696502 0.19779372 0.08920058 0.27405855 0.16721794 0.10244921
 0.18454795 0.32990026 0.19594619 0.19614896 0.2741876  0.11739593
 0.18954574 0.22440885 0.22733237 0.15061055 0.11048189 0.07654032
 0.14093786 0.15809296 0.11600715 0.07401273 0.21971506 0.08143677
 0.09671821 0.19747443 0.09868856 0.10392119 0.13555452 0.21118957
 0.30183163 0.47138    0.08099389 0.13463983 0.24988116 0.25630713
 0.11776771 0.12669826 0.15278456 0.0486639  0.0767245  0.13649915
 0.16960455 0.1340462  0.157601   0.22796819 0.09766578 0.23822698
 0.04696283 0.13323571 0.14116673 0.24511705 0.16408749 0.14867207
 0.168294

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,